In [1]:
import pymongo
import time
import threading
import random
import string
from pymongo.collection import Collection

In [2]:
cl = pymongo.MongoClient("mongodb://localhost:27017")
db = cl["deep"]
obj1 = db["obj1"]
obj2 = db["obj2"]
obj4 = db["obj4"]
obj8 = db["obj8"]
coll_list = [obj1, obj2, obj4, obj8]
user_list = [1, 2, 4]
depth_list = [1, 2, 4, 8]

In [3]:
l1_device_field = "l1.device"
l2_device_field = "l1.l2.device"
l4_device_field = "l1.l2.l3.l4.device"
l8_device_field = "l1.l2.l3.l4.l5.l6.l7.l8.device"
device_fields = [l1_device_field, l2_device_field, l4_device_field, l8_device_field]

In [4]:
l1_vol_field = "l1.total_volume_bytes"
l2_vol_field = "l1.l2.total_volume_bytes"
l4_vol_field = "l1.l2.l3.l4.total_volume_bytes"
l8_vol_field = "l1.l2.l3.l4.l5.l6.l7.l8.total_volume_bytes"
vol_fields = [l1_vol_field, l2_vol_field, l4_vol_field, l8_vol_field]

In [5]:
l1_app_field = "l1.app"
l2_app_field = "l1.l2.app"
l4_app_field = "l1.l2.l3.l4.app"
l8_app_field = "l1.l2.l3.l4.l5.l6.l7.l8.app"
app_fields = [l1_app_field, l2_app_field, l4_app_field, l8_app_field]

In [6]:
l1_device_path = ["l1", "device"]
l2_device_path = ["l1", "l2", "device"]
l4_device_path = ["l1", "l2", "l3", "l4", "device"]
l8_device_path = ["l1", "l2", "l3", "l4", "l5", "l6", "l7", "l8", "device"]
device_paths =[l1_device_path, l2_device_path, l4_device_path, l8_device_path]

In [7]:
l1_vol_path = ["l1", "total_volume_bytes"]
l2_vol_path = ["l1", "l2", "total_volume_bytes"]
l4_vol_path = ["l1", "l2", "l3", "l4", "total_volume_bytes"]
l8_vol_path = ["l1", "l2", "l3", "l4", "l5", "l6", "l7", "l8", "total_volume_bytes"]
vol_paths =[l1_vol_path, l2_vol_path, l4_vol_path, l8_vol_path]

In [8]:
def random_word(length):
    return ''.join(random.choices(string.ascii_lowercase, k=length))

In [9]:
def indexedWhereClause(user_id, depth):
    field = device_fields[depth]
    query = {field: user_id}
    return query

In [10]:
def nonIndexedWhereClause(user_id, depth):
    field = vol_fields[depth]
    query = {field: user_id}
    return query

In [12]:
def indexedUpdate(depth):
    field = device_fields[depth]
    query = [{"$set": {field: random.randint(0,9999)}}]
    return query

In [102]:
def nonIndexedUpdate(depth):
    field = vol_fields[depth]
    query = [{"$set": {field: random.randint(0,9999)}}]
    return query

In [42]:
def resolve_where(user_id, depth, isWhereIndex):
    if isWhereIndex:
        return indexedWhereClause(user_id, depth)
    else:
        return nonIndexedWhereClause(user_id, depth)

In [43]:
def resolve_update(depth, isUpdateIndex):
    if isUpdateIndex:
        return indexedUpdate(depth)
    else:
        return nonIndexedUpdate(depth)

In [15]:
def delete_overlapping():
    for d in range(len(depth_list)):
        coll = coll_list[d]
        device_field = device_fields[d]
        vol_field = vol_fields[d]
        print(coll.delete_many({device_field: {"$lte": 4}}))
        print(coll.delete_many({vol_field: {"$lte": 4}}))

In [16]:
def delete_mocks():
    for d in range(len(depth_list)):
        coll = coll_list[d]
        print(coll.delete_many({"created": True}))

In [17]:
def count_mocks():
    for d in range(len(depth_list)):
        coll = coll_list[d]
        device_field = device_fields[d]
        vol_field = vol_fields[d]
        device_count = coll.count_documents({device_field: {"$lte": 4}})
        vol_count = coll.count_documents({vol_field: {"$lte": 4}})
        print(f"depth = {depth_list[d]}, device = {device_count}, vol = {vol_count}")

In [106]:
def count_created():
    for d in range(len(depth_list)):
        coll = coll_list[d]
        print(coll.count_documents({"created": True}))

In [18]:
def create_device_mocks(user_id: int, size: int, depth: int, coll: Collection):
    offset = random.randint(0, 9000)
    data = list(coll.find({}, {"_id": 0}).skip(offset).limit(size))
    path = device_paths[depth]
    for d in data:
        d["created"] = True
        for key in path[:-1]:
            d = d[key]
        d[path[-1]] = user_id
    return data

In [19]:
def create_vol_mocks(user_id: int, size: int, depth: int, coll: Collection):
    offset = random.randint(0, 9000)
    data = list(coll.find({}, {"_id": 0}).skip(offset).limit(size))
    path = vol_paths[depth]
    for d in data:
        d["created"] = True
        for key in path[:-1]:
            d = d[key]
        d[path[-1]] = user_id
    return data

In [20]:
def insert_mocks(size: int):
    print("inserting mocks")
    for d in range(len(depth_list)):
        coll = coll_list[d]
        print(f"  depth {depth_list[d]}:", end=" ")
        print("user - ", end="")
        for u in range(user_list[-1]):
            device_mocks = create_device_mocks(u, size, d, coll)
            vol_mocks = create_vol_mocks(u, size, d, coll)
            coll.insert_many(device_mocks)
            coll.insert_many(vol_mocks)
            print(f"{u}", end=" ")
        print("---")

In [24]:
def createIndexes():
    obj1.create_index([("l1.device", 1)])
    obj2.create_index([("l1.l2.device", 1)])
    obj4.create_index([("l1.l2.l3.l4.device", 1)])
    obj8.create_index([("l1.l2.l3.l4.l5.l6.l7.l8.device", 1)])

In [25]:
def clearIndexes():
    obj1.drop_indexes()
    obj2.drop_indexes()
    obj4.drop_indexes()
    obj8.drop_indexes()

In [26]:
def showIndexes():
    for c in coll_list:
        for i in c.list_indexes():
            print(i)

In [27]:
def calculate_diffs(old, new):
    for u in range(len(user_list)):
        print(f"user {user_list[u]}")
        for d in range(len(depth_list)):
            o_val = old[u][d]
            n_val = new[u][d]
            diff = ((n_val - o_val) / o_val) * 100
            print(f"| {depth_list[d]} |{o_val:10.0f}| {n_val:10.0f}| {diff:10.1f} |")
        print("--------")

In [147]:
def reset():
    delete_mocks()
    insert_mocks(500)

In [49]:
def measure_update(user_id, duration_list, coll: Collection, depth, isWhereIndex, isUpdateIndex):
    where = resolve_where(user_id, depth, isWhereIndex)
    pipe = resolve_update(depth, isUpdateIndex)
    
    start = time.perf_counter()
    coll.update_many(where, pipe)
    end = time.perf_counter()
    duration = (end - start) * 1000

    duration_list[user_id] = duration

In [47]:
def test_update(num_of_users, depth, isWhereIndex, isUpdateIndex):
    threads_list = []
    duration_list = [None] * num_of_users
    coll = coll_list[depth]

    for user_id in range(num_of_users):
        t = threading.Thread(target=measure_update, args=(user_id, duration_list, coll, depth, isWhereIndex, isUpdateIndex))
        t.start()
        threads_list.append(t)

    
    for t in threads_list:
        t.join()
        
    result = sum(duration_list) / len(duration_list)
    return result    

In [52]:
def execute_update(isWhereIndex, isUpdateIndex):
    u_len = len(user_list)
    d_len = len(depth_list)
    result = [[0 for _ in range(d_len)] for _ in range(u_len)]
    
    for u in range(u_len):
        print(f"user - {user_list[u]}")
        print("  depth", end=" ")
        for d in range(d_len):
            result[u][d] = test_update(user_list[u], d, isWhereIndex, isUpdateIndex)
            print(f"{depth_list[d]}", end=" ")

        print("---")
    return result

In [148]:
clearIndexes()

In [149]:
showIndexes()

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])


In [150]:
reset()

DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
inserting mocks
  depth 1: user - 0 1 2 3 ---
  depth 2: user - 0 1 2 3 ---
  depth 4: user - 0 1 2 3 ---
  depth 8: user - 0 1 2 3 ---


In [151]:
without_index = execute_update(True, True)

user - 1
  depth 1 2 4 8 ---
user - 2
  depth 1 2 4 8 ---
user - 4
  depth 1 2 4 8 ---


In [152]:
reset()

DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
inserting mocks
  depth 1: user - 0 1 2 3 ---
  depth 2: user - 0 1 2 3 ---
  depth 4: user - 0 1 2 3 ---
  depth 8: user - 0 1 2 3 ---


In [153]:
createIndexes()

In [154]:
count_mocks()

depth = 1, device = 2000, vol = 2000
depth = 2, device = 2000, vol = 2000
depth = 4, device = 2000, vol = 2000
depth = 8, device = 2000, vol = 2000


In [155]:
where_index_update_non_index = execute_update(True, False)

user - 1
  depth 1 2 4 8 ---
user - 2
  depth 1 2 4 8 ---
user - 4
  depth 1 2 4 8 ---


In [159]:
reset()

DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
inserting mocks
  depth 1: user - 0 1 2 3 ---
  depth 2: user - 0 1 2 3 ---
  depth 4: user - 0 1 2 3 ---
  depth 8: user - 0 1 2 3 ---


In [160]:
clearIndexes()

In [161]:
createIndexes()

In [162]:
where_non_index_update_index = execute_update(False, True)

user - 1
  depth 1 2 4 8 ---
user - 2
  depth 1 2 4 8 ---
user - 4
  depth 1 2 4 8 ---


In [165]:
clearIndexes()

In [166]:
reset()

DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
inserting mocks
  depth 1: user - 0 1 2 3 ---
  depth 2: user - 0 1 2 3 ---
  depth 4: user - 0 1 2 3 ---
  depth 8: user - 0 1 2 3 ---


In [167]:
createIndexes()

In [168]:
where_index_update_index = execute_update(True, True)

user - 1
  depth 1 2 4 8 ---
user - 2
  depth 1 2 4 8 ---
user - 4
  depth 1 2 4 8 ---


In [169]:
clearIndexes()

In [170]:
reset()

DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 4000, 'ok': 1.0}, acknowledged=True)
inserting mocks
  depth 1: user - 0 1 2 3 ---
  depth 2: user - 0 1 2 3 ---
  depth 4: user - 0 1 2 3 ---
  depth 8: user - 0 1 2 3 ---


In [171]:
createIndexes()

In [172]:
calculate_diffs(without_index, where_index_update_index)

user 1
| 1 |        83|         22|      -73.5 |
| 2 |        82|         24|      -70.6 |
| 4 |       100|         26|      -74.2 |
| 8 |       122|         29|      -76.0 |
--------
user 2
| 1 |       118|         10|      -91.9 |
| 2 |       131|         11|      -91.3 |
| 4 |       110|         15|      -85.9 |
| 8 |       134|         17|      -87.7 |
--------
user 4
| 1 |       170|         12|      -92.8 |
| 2 |       163|         16|      -89.9 |
| 4 |       173|         13|      -92.5 |
| 8 |       193|         23|      -88.2 |
--------


In [173]:
calculate_diffs(without_index, where_index_update_non_index)

user 1
| 1 |        83|         21|      -74.4 |
| 2 |        82|         17|      -78.8 |
| 4 |       100|         16|      -83.5 |
| 8 |       122|         22|      -82.3 |
--------
user 2
| 1 |       118|         21|      -82.2 |
| 2 |       131|         24|      -81.4 |
| 4 |       110|         34|      -69.3 |
| 8 |       134|         24|      -82.4 |
--------
user 4
| 1 |       170|         24|      -85.8 |
| 2 |       163|         31|      -80.8 |
| 4 |       173|         37|      -78.6 |
| 8 |       193|         50|      -74.2 |
--------


In [174]:
calculate_diffs(without_index, where_non_index_update_index)

user 1
| 1 |        83|         90|        9.6 |
| 2 |        82|        104|       25.8 |
| 4 |       100|        120|       20.3 |
| 8 |       122|        133|        9.3 |
--------
user 2
| 1 |       118|        154|       30.2 |
| 2 |       131|        134|        2.4 |
| 4 |       110|        112|        2.2 |
| 8 |       134|        162|       20.9 |
--------
user 4
| 1 |       170|        180|        5.8 |
| 2 |       163|        183|       12.2 |
| 4 |       173|        203|       17.0 |
| 8 |       193|        271|       40.4 |
--------
